In [1]:
import findspark
findspark.init()


In [2]:
import time
from sys import argv

from helpers.config import config
from helpers.ioutil import download_data, put_file_in_hdfs
from helpers.sparkutil import SparkUtil
from constants.module_constants import ModuleConstants



In [3]:
local_path = 'data_notebook.csv'
download_data('https://chronicdata.cdc.gov/views/735e-byxc/rows.csv?accessType=DOWNLOAD', local_path)
hdfs_file_path = config()['FILE'][ModuleConstants.HADOOP_FILE_COPY_PATH] + 'data_' +\
                 str(int(round(time.time() * 1000))) + '.csv'
print(hdfs_file_path)

hdfs://localhost:9000/data_1586586931606.csv


In [4]:
put_file_in_hdfs(local_path, hdfs_file_path)

Running shell command: hdfs dfs -put data_notebook.csv hdfs://localhost:9000/data_1586586931606.csv
0 b'' b''


In [5]:
batch_data_frame = SparkUtil.load_data("com.databricks.spark.csv", hdfs_file_path)

In [6]:
batch_data_frame = batch_data_frame \
    .withColumnRenamed("High_Confidence_Limit ", "High_Confidence_Limit") \
    .withColumnRenamed("Age(months)", "Age")
batch_data_frame.describe()

DataFrame[summary: string, YearStart: string, YearEnd: string, LocationAbbr: string, LocationDesc: string, Datasource: string, Class: string, Topic: string, Question: string, Data_Value_Unit: string, Data_Value_Type: string, Data_Value: string, Data_Value_Alt: string, Data_Value_Footnote_Symbol: string, Data_Value_Footnote: string, Low_Confidence_Limit: string, High_Confidence_Limit: string, Sample_Size: string, Total: string, Age: string, Gender: string, Race/Ethnicity: string, GeoLocation: string, ClassID: string, TopicID: string, QuestionID: string, DataValueTypeID: string, LocationID: string, StratificationCategory1: string, Stratification1: string, StratificationCategoryId1: string, StratificationID1: string]

In [7]:
batch_data_frame.write.mode("overwrite").saveAsTable(config()['HIVE'][ModuleConstants.HIVE_APP_TABLE])

In [8]:
query = 'select count(*) from ' + config()['HIVE'][ModuleConstants.HIVE_APP_TABLE]
SparkUtil.execute_show_query(query)


+--------+
|count(1)|
+--------+
|    9180|
+--------+



In [9]:
queries = config()['QUERIES'][ModuleConstants.QUERIES].split('^')
tabales = config()['QUERIES'][ModuleConstants.RESULT_TABLES].split(',')
SparkUtil.execute_quey_store_results(queries, tabales)


running query :: select 'ALL' AS category,questionid,yearstart AS year,avg(data_value) AS avarage from servay2020 group by questionid,yearstart order by questionid,yearstart  , storing results in :: servay_analysis_all_results
+--------+----------+----+------------------+
|category|questionid|year|           avarage|
+--------+----------+----+------------------+
|     ALL|      Q040|2008| 16.37172413793103|
|     ALL|      Q040|2010|16.231841652323595|
|     ALL|      Q040|2012|15.599483648881238|
|     ALL|      Q040|2014|15.436269430051802|
|     ALL|      Q040|2016|15.148615916955029|
|     ALL|      Q041|2008|14.983793103448296|
|     ALL|      Q041|2010|15.064716006884682|
|     ALL|      Q041|2012|14.276592082616187|
|     ALL|      Q041|2014|13.943350604490504|
|     ALL|      Q041|2016| 13.53235294117647|
|     ALL|      Q060|2008|13.218181818181813|
|     ALL|      Q060|2010|13.046984126984121|
|     ALL|      Q060|2012|11.963057324840753|
|     ALL|      Q060|2014|11.45063897

In [10]:
SparkUtil.spark_session().stop()